In [ ]:
import rasterio
from pathlib import Path
from PIL import Image
import numpy as np
import ipyplot
import json

# Questions
* there is no sun elevation from satellogic ... right?
* what is the link between View Incidence Angle and off-nadir angle. Would it be possible to put in same format?


# Changes
* use red.tif green.tif ... for satellogic names
* name the current angles in a sub-section call View Incidence Angle
* Could we also extract the sun angles from the xml


In [ ]:
def load_band(file_path):
    with rasterio.open(file_path) as src:
        tags = src.tags()
        data = src.read()

    return data, tags

def load_rgb_from_bands(image_dir, band_names = ("B02.tif", "B03.tif", "B04.tif")):

    RGB = []
    for file_name in band_names:
        data, tags = load_band(image_dir / file_name)
        RGB.append(data)

    RGB = np.moveaxis(np.stack(RGB, axis=0)[:,0,:,:], 0,2)
    return float_image_to_uint8([RGB])[0]

# cut-n-paste from the benchmark code
def float_image_to_uint8(images, percentile_max=99.9, ensure_3_channels=True, per_channel_scaling=False):
    """Convert a batch of images to uint8 such that 99.9% of values fit in the range (0,255)."""
    images = np.asarray(images)
    if images.dtype == np.uint8:
        return images

    images = images.astype(np.float64)

    if per_channel_scaling:
        mx = np.percentile(images, q=percentile_max, axis=(0, 1, 2), keepdims=True)
        mx = np.squeeze(mx, axis=0)
        mn = np.percentile(images, q=100 - percentile_max, axis=(0, 1, 2), keepdims=True)
    else:
        mn = np.percentile(images, q=100 - percentile_max)
        mx = np.percentile(images, q=percentile_max)

    new_images = []
    for image in images:
        image = np.clip((image - mn) * 255 / (mx - mn), 0, 255)
        if ensure_3_channels:
            if image.ndim == 2:
                image = np.stack((image, image, image), axis=2)
            if image.shape[2] == 1:
                image = np.concatenate((image, image, image), axis=2)
        new_images.append(image.astype(np.uint8))
    return new_images


In [ ]:
satellogic_dir = Path("/mnt/data/unsupervised_dataset/small_dataset_sample/satellogic/zip_1/34_04N115_86W/2021-11-19")

RGB_img = load_rgb_from_bands(satellogic_dir)
RGB_enhanced = np.asarray(Image.open(satellogic_dir / "RGB_enhanced.jpg"))
NIR_img, _ = load_band(satellogic_dir / "B08.tif")
NIR_img = float_image_to_uint8([NIR_img[0]])[0]

with open(satellogic_dir / "metadata.json", 'r') as fd:
    satellogic_metadata = json.load(fd)

print("RGB_img: ", RGB_img.shape,RGB_img.dtype)
print("RGB_enh: ", RGB_enhanced.shape, RGB_enhanced.dtype)
print("NIR_img: ", NIR_img.shape, NIR_img.dtype)


images = [RGB_img, RGB_enhanced, NIR_img]
labels = ["RGB_img", "RGB_enhanced", "NIR_img"]

In [ ]:
ipyplot.plot_class_tabs(images=images, labels=labels, img_width=500, max_imgs_per_tab=10)



In [ ]:
sentinel_dir = Path("/mnt/data/unsupervised_dataset/small_dataset_sample/sentinel_2/zip_0/33_70N116_86W")

images = []
labels = []
for time_dir in sentinel_dir.iterdir():
    band_dict = {}
    for band_path in time_dir.glob("*.tif"):
        
        band_name = band_path.stem
        band_data, _ = load_band(band_path)
                
        if band_name == "SCL":
            band_data *= 20
        band_dict[band_name] = band_data[0]
        
    with open(time_dir / "metadata.json", 'r') as fd:
        sentinel_metadata = json.load(fd)

        
    keys = sorted(band_dict.keys())
    for key in keys:
        labels.append(key)
        band_img = float_image_to_uint8([band_dict[key]])[0]
        images.append(band_img)
        
    RGB_img = np.stack([band_dict[key] for key in ("B04", "B03", "B02")], axis=2)
    RGB_img = float_image_to_uint8([RGB_img])[0]
    images.append(RGB_img)
    labels.append("RGB")

In [ ]:
ipyplot.plot_class_tabs(images=images, labels=labels, img_width=400, max_imgs_per_tab=10)


In [ ]:
satellogic_metadata

In [ ]:
sentinel_metadata